# Differential Evolution

In [624]:
import numpy as np
from numpy.random import choice, uniform

In [625]:
def init_population(pop_size, var_count, min_bound, max_bound):
    return uniform(low=min_bound, high=max_bound, size=(pop_size, var_count))

In [626]:
population = init_population(10, 2, -30, 30)
population

array([[ 13.77893293, -17.99298165],
       [-29.48295903, -11.19645268],
       [ -1.15731556, -22.11733265],
       [ 24.16982069, -16.05543582],
       [-27.56786659,  -0.04061984],
       [-21.38970959, -25.25397115],
       [  6.37249628, -11.04256388],
       [ 26.04539319,  -7.3587193 ],
       [-16.8912084 ,  -3.17605325],
       [ 10.2456691 ,  23.55973209]])

In [627]:
def calc_fitness_scores(population):
    scores = []

    for vector in population:
        # Rosenbrock's Function
        score = 100 * (vector[1] - vector[0]**2)**2 + (vector[0] - 1)**2
        scores.append(score)

    return scores

In [628]:
scores = calc_fitness_scores(population)
scores

[4320407.62726378,
 77518622.00786187,
 55026.38760135734,
 36028822.55693385,
 57765055.17904898,
 23307540.790834118,
 266814.260577627,
 47021987.17723408,
 8322908.941737417,
 662909.4732217855]

In [629]:
def differential_mutation(idx, population, min_bound, max_bound, scaling_factor = 0.5):
    used_idx = [idx]
    used_idx_no = 3

    while len(used_idx) <= used_idx_no:
        idx_selected = False
        while idx_selected == False:
            vector_idx = choice(np.array(range(len(population))), 1, replace=False)[0]
            if vector_idx not in used_idx:
                idx_selected = True
                used_idx.append(vector_idx)

    target_vector = used_idx[1]
    ran_select_1 = used_idx[2]
    ran_select_2 = used_idx[3]

    trial_vector = []
    trial_vec_loc_1 = population[target_vector][0] + scaling_factor * (population[ran_select_1][0] - population[ran_select_2][0])
    if trial_vec_loc_1 < min_bound:
        trial_vector.append(min_bound)
    elif trial_vec_loc_1 > max_bound:
        trial_vector.append(max_bound)
    else:
        trial_vector.append(trial_vec_loc_1)
    trial_vec_loc_2 = population[target_vector][1] + scaling_factor * (population[ran_select_1][1] - population[ran_select_2][1])
    if trial_vec_loc_2 < min_bound:
        trial_vector.append(min_bound)
    elif trial_vec_loc_2 > max_bound:
        trial_vector.append(max_bound)
    else:
        trial_vector.append(trial_vec_loc_2)

    return trial_vector

In [630]:
trial_vector = differential_mutation(1, population, -30, 30)
trial_vector

[-30, 0.482996485872178]

In [631]:
def crossover(idx, population, trial_vector, crossover_prob = 0.7):
    parent_vector = population[idx]
    offspring_vector = []
    var_len = len(parent_vector)
    trial_vec_select_idx = list(choice(np.array(range(var_len)), 1, replace=False))
    for i in range(var_len):
        if uniform(0, 1) < crossover_prob and i not in trial_vec_select_idx:
            trial_vec_select_idx.append(i)
    for j in range(var_len):
        if j in trial_vec_select_idx:
            offspring_vector.append(trial_vector[j])
        else:
            offspring_vector.append(parent_vector[j])

    return offspring_vector

In [632]:
offspring_vector = crossover(1, population, trial_vector, crossover_prob = 0.7)
offspring_vector

[-30, -11.196452680054925]

In [633]:
def selection(idx, offspring_vector, population, scores):
    sub_pop = [population[idx], offspring_vector]
    sub_scores = calc_fitness_scores(sub_pop)
    if sub_scores[1] < sub_scores[0]:
        population[idx] = offspring_vector
        scores[idx] = sub_scores[1]

    return

In [634]:
selection(1, offspring_vector, population, scores)
population

array([[ 13.77893293, -17.99298165],
       [-29.48295903, -11.19645268],
       [ -1.15731556, -22.11733265],
       [ 24.16982069, -16.05543582],
       [-27.56786659,  -0.04061984],
       [-21.38970959, -25.25397115],
       [  6.37249628, -11.04256388],
       [ 26.04539319,  -7.3587193 ],
       [-16.8912084 ,  -3.17605325],
       [ 10.2456691 ,  23.55973209]])

In [635]:
scores

[4320407.62726378,
 77518622.00786187,
 55026.38760135734,
 36028822.55693385,
 57765055.17904898,
 23307540.790834118,
 266814.260577627,
 47021987.17723408,
 8322908.941737417,
 662909.4732217855]

In [636]:
def differential_evolution(var_count, min_bound, max_bound, pop_size, generations, scaling_factor = 0.5, crossover_prob = 0.7):
    population = init_population(pop_size, var_count, min_bound, max_bound)
    scores = calc_fitness_scores(population)
    for i in range(generations):
        print(f"******************************* Generation {i + 1} *******************************")
        print("Population-------------------------------------------------------Scores")
        for idx, vector in enumerate(population):
            trial_vector = differential_mutation(idx, population, min_bound, max_bound)
            offspring_vector = crossover(idx, population, trial_vector, 0.7)
            selection(idx, offspring_vector, population, scores)
            print(str(population[idx])+"                              "+str(scores[idx]))
        print("***************************************************************************")

In [637]:
differential_evolution(2, -30, 30, 10, 10)

******************************* Generation 1 *******************************
Population-------------------------------------------------------Scores
[18.29935502  9.89839763]                              10560719.133462628
[ 3.04545064 -1.48868499]                              11589.379392961357
[-14.13093721 -30.        ]                              5275674.748417325
[-18.37434458 -20.43692393]                              12820603.585464254
[  1.75467656 -12.91909934]                              25594.135300765793
[-10.10200536  -9.02817544]                              1233970.7392566933
[0.75174841 0.17013492]                              15.663398542831468
[12.13346416  2.33724131]                              2099252.9325266783
[-7.48553967 18.02395085]                              144543.0983168521
[12.66142615 -4.69396265]                              2722823.2578281267
***************************************************************************
*******************************